In [ ]:
from google.colab import drive
import json
import random
import requests

drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Define the path to your dataset in Google Drive
json_path = "/content/drive/MyDrive/Colab Notebooks/PFA/multi_label_val.json"

# Load the JSON file
with open(json_path, 'r') as f:
    data = json.load(f)²

print(f"Loaded {len(data)} entries from the dataset.")

In [ ]:
# Load the dataset (choose the appropriate file)
dataset_file = "/content/drive/MyDrive/Colab Notebooks/PFA/multi_label_val.json"  # or "eccv_train.json"
with open(json_path, 'r') as f:
    raw_data = f.read(500)  # Read first 500 characters
    print(raw_data)

with open(json_path, 'r') as f:
    data = json.load(f)

print(data.keys())  # Print the top-level keys

In [ ]:
# Define the target categories
target_categories = {"on fire", "earthquake", "heavy rainfall", "fog"}

# Dictionary to store selected images per category
selected_images = {category: [] for category in target_categories}

# Function to check if a URL is active
def is_url_active(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False


In [ ]:
# Iterate through the dataset and collect images
for filename, metadata in data.items():
    incidents = metadata.get("incidents", {})  # Get the "incidents" dictionary
    labels = set(incidents.keys())  # Extract label names

    # Find intersection with target categories
    common_labels = labels.intersection(target_categories)

    if common_labels:
        for label in common_labels:
            if len(selected_images[label]) < (1000 // len(target_categories)):
                image_url = metadata.get("url")
                if is_url_active(image_url):  # Check if the URL is active
                    selected_images[label].append({
                        "filename": filename,
                        "url": image_url,
                        "category": label
                    })

# Flatten the dictionary into a single list
final_subset = [img for images in selected_images.values() for img in images]

# Shuffle the dataset to ensure randomness
random.shuffle(final_subset)

print(f"Total selected images: {len(final_subset)}")

In [ ]:
# Save the cleaned subset
subset_file = "incident_subset_1000.json"
with open(subset_file, 'w') as f:
    json.dump(final_subset, f, indent=4)

print(f"Subset saved as {subset_file}")